In [10]:
import talib
# import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import datetime
import MetaTrader5 as mt5

In [22]:
# MT5 connection

if not mt5.initialize(login=114781990, server="Exness-MT5Trial6",password="Mypassword$1234"):
            print("initialize() failed, error code =",mt5.last_error())
            quit()
        
login = 114781990
password = 'Mypassword$1234'
server = 'Exness-MT5Trial6'

mt5.login(login, password, server)

True

In [27]:
# Downloading data and creating two datasets

rate = mt5.copy_rates_from('EURUSDm', mt5.TIMEFRAME_M15, datetime.datetime.now(), 24000)
nf = pd.DataFrame(rate)
        
         
nf['date']=pd.to_datetime(nf['time'], unit='s')

nf.set_index(np.arange(len(nf)), inplace = True)
nf = nf.drop(['spread', 'real_volume', 'tick_volume', 'time'], axis='columns')


df = nf.iloc[:500]  # First half
df2 = nf.iloc[500:]  # Second half
df['squeeze_on'] = False



C:\Users\hp\AppData\Local\Temp\ipykernel_8680\1789775042.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['squeeze_on'] = False


In [38]:
# Indicators

# Creating indicators code, 
# I need 200 candles to create EMA_200
# 20 candles for keltners, 21 candles for RSI

# def in_squeeze(self, df):
#     return df['lower_band'] > df['lower_keltner'] and df['upper_band'] < df['upper_keltner']


def get_kc( high, low, close, kc_lookback, multiplier, atr_lookback):
    tr1 = pd.DataFrame(high - low)
    tr2 = pd.DataFrame(abs(high - close.shift()))
    tr3 = pd.DataFrame(abs(low - close.shift()))

    # Calculate True Range as the element-wise maximum of the three components
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    atr = tr.ewm(alpha=1/atr_lookback).mean()

    kc_middle = close.ewm(span=kc_lookback).mean()
    kc_upper = kc_middle + multiplier * atr
    kc_lower = kc_middle - multiplier * atr

    return kc_middle.iloc[-1], kc_upper.iloc[-1], kc_lower.iloc[-1]



def calculate_indicators():
    '''Adding EMAs, BB, Keltner Channel and RSI'''

#     df['EMA_8'] = talib.EMA(df['close'], timeperiod=8)
    
    global df

    length = len(df) - 1
    
    df.loc[length, 'EMA_8'] = talib.EMA(df['close'], timeperiod=8).iloc[-1]

    
    df.loc[length,'EMA_200'] = talib.EMA(df['close'], timeperiod=200).iloc[-1]

    # RSI
    df.loc[length,'RSI'] = talib.RSI(df['close'], timeperiod=21).iloc[-1]


    # Identify trend on 21 days highs & lows
#     df = get_trend(df)

    '''Bollinger Band'''
    
    sma = df['close'].rolling(window=20).mean()
    stddev = df['close'].rolling(window=20).std()
    lower_band = sma - (2 * stddev)
    upper_band = sma + (2 * stddev)
    
#     df['20sma'] = df['close'].rolling(window=20).mean()
#     df['stddev'] = df['close'].rolling(window=20).std()
#     df['lower_band'] = df['20sma'] - (2 * df['stddev'])
#     df['upper_band'] = df['20sma'] + (2 * df['stddev'])

#     Keltner 1.2 Multiplier for squeeze
    
    kc_middle, kc_upper, kc_lower = get_kc(df['high'], df['low'], df['close'], kc_lookback=20, multiplier=1.2, atr_lookback=10)
    
#     Write the squeeze logic here
#     No need to add it to df

#     print(lower_band.iloc[-1], upper_band.iloc[-1])

    if (lower_band.iloc[-1] > kc_lower) and (upper_band.iloc[-1] < kc_upper):
        
        df.loc[length,'squeeze_on'] = True
        
    else:
        
        df.loc[length,'squeeze_on'] = False

#     df['squeeze_on'] = df.apply(in_squeeze, axis=1)

#     df.drop(['stddev', 'lower_band', 'upper_band', '20sma', 'TR', 'ATR', 'lower_keltner', 'upper_keltner'], axis=1, inplace=True)

    '''Get Keltner for trading'''
    df.loc[length,'EMA_20'], df.loc[length,'upper_keltner'], df.loc[length,'lower_keltner'] = get_kc(df['high'], df['low'], df['close'], kc_lookback=20, multiplier=2.25, atr_lookback=10)
    
    
#     df.dropna(inplace=True)
#     df.reset_index(drop=True, inplace=True)
#     df.index.name = 'sn'
    
    

In [39]:

global df, df2
# Initialize an empty DataFrame to store the results
result_df = pd.DataFrame()

# Initialize an empty DataFrame to store the past data that gets dropped
past_df = pd.DataFrame()

for index, row in df2.iterrows():
    
    # Add live data to the df DataFrame
    df = df.append(row, ignore_index=True)
#     print(len(df))
    # Delete the first row if the DataFrame has more than 202 rows
    if len(df) > 500:
        # Save the dropped row in past_df
        past_df = past_df.append(df.iloc[0], ignore_index=True)
        df = df.iloc[1:].reset_index(drop=True)
        
    # Calculate indicators for the current state of df
    calculate_indicators()
    
# Concatenate past_df and df to get the full dataset
result_df = pd.concat([past_df, df], ignore_index=True)

# Save the result_df to an Excel file
result_df.to_excel('indicator_check.xlsx')


C:\Users\hp\AppData\Local\Temp\ipykernel_8680\2309130152.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_8680\2309130152.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  past_df = past_df.append(df.iloc[0], ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_8680\2309130152.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_8680\2309130152.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  past_df = past_df.append(df.iloc[0], ignore_index=True)
C:\Users\hp\Ap